In [1]:
# https://cathiesark.com/arkk/trends

import sys
print(sys.version)

import tdameritrade as td # using local installation
import tdameritrade.auth
import numpy as np
import pandas as pd
import traceback
from option_price import OptionPrice
from datetime import datetime, timedelta, date

from yahoo_earnings_calendar import YahooEarningsCalendar
yec = YahooEarningsCalendar()

pd.set_option('display.max_rows', 50)


"""
https://swingtradebot.com/
https://stockgains.io/
"""
init = False
if not('token' in globals()): 
    token = None


print('ready')

OPTION_CALL = 'CALL'
OPTION_PUT = 'PUT'

3.7.11 (default, Jul  6 2021, 12:43:19) 
[Clang 12.0.5 (clang-1205.0.22.9)]
ready


In [2]:
""" potential buy
PAYO
DOCN
SFT
SHAK
SPLK
SPCE
WKME
DOCN
PUBM
AI
MDLA
"""

""" potential option
ZUO
PD
FSR
WMT
COF
SQ
JPM
V
NIO
WKHS

AKAM
WFC
NFLX
SRPT
BA
FB
BL
GDX
SLV
ANGI
crm
UBER
AKAM
CSOD
WMT
NET
DKNG
GOOG
NRZ
TWLO
WORK
XOM
FOUR
SONO
ea
PLT
HUBS
PRPL
SNE
SPOT
GOOG
CTOY
ZEN
MSFT
FSLY
PLAY
AAL
"""

' potential option\nZUO\nPD\nFSR\nWMT\nCOF\nSQ\nJPM\nV\nNIO\nWKHS\n\nAKAM\nWFC\nNFLX\nSRPT\nBA\nFB\nBL\nGDX\nSLV\nANGI\ncrm\nUBER\nAKAM\nCSOD\nWMT\nNET\nDKNG\nGOOG\nNRZ\nTWLO\nWORK\nXOM\nFOUR\nSONO\nea\nPLT\nHUBS\nPRPL\nSNE\nSPOT\nGOOG\nCTOY\nZEN\nMSFT\nFSLY\nPLAY\nAAL\n'

In [3]:
# add token
print(init, token)
if init or token:
    c = td.TDClient(client_id='USS4AQEULHOTQVMHZPWG0JU6QHFPIUU7', refresh_token=token)
    print('init client')
    
# SYMS = ['DKNG'] # UPWK
SYMS=[''] # ['SUMO']
BUYING=0 # False to indicate best options to sell covered calls on.
OUT_THE_MONEY=0 or not BUYING
OPTION_TYPE = OPTION_CALL
LONG_TERM = False

False None


In [4]:
# https://tdameritrade.readthedocs.io/en/latest/index.html#authentication
# token

In [5]:
MAX_SPREAD=.4
RFR = .0061 # 10y risk free rate


In [6]:
class Option:
    
    def __init__(self, sym, df, buying=True, out_the_money=True, option_type=OPTION_CALL):
        self.df = df
        self.sym = sym
        self.buying = buying
        self.out_the_money = out_the_money
        self.option_type = option_type
        print('option', sym, buying, out_the_money, option_type)
        
    def get_standard(self):
        return self.df[self.df['nonStandard'] == True]
        
    def get_for_date(self, month, day, year=2020):
        d = '{}-{:02d}-{:02d} 01:00:00'.format(year, month, day)
        print('d', d)
        return self.df[self.df['expirationDate'] == pd.Timestamp(np.datetime64(d))]
    
    def get_after_date(self, month, day, year=2020):
        d = '{}-{:02d}-{:02d} 01:00:00'.format(year, month, day)
        print('d', d)
        return self.df[self.df['expirationDate'] >= pd.Timestamp(np.datetime64(d))]
    
    def sort(self, col, asc=False):
        return self.df.sort_values(by=col, ascending=asc)

    def last_price(self):
        return c.quote(sym)['lastPrice']
    
    def info(self):
        print(self.df.columns.values)
        print(self.df.head())
        print(self.df.info())
    
    def process_data(self):
        df = self.get_after_date(5,1)
        print(df.shape)
        df = df[df['putCall'] == self.option_type]
        # TODO
        
        IS_CALL = self.option_type == OPTION_CALL
        
        df['break_even'] = df['ask'] + df['strikePrice'] if IS_CALL else df['strikePrice'] - df['ask']
        df['spread'] = df['ask'] - df['bid']
        df['spread_ratio'] = df['spread'] / df['ask']
        df = df[(df['bid'] >= 0) & (df['spread_ratio'] < MAX_SPREAD) & (df['last'] >= 0)]
        # df = df[(df['totalVolume'] >= 0)]
        
        last_price = c.quote(self.sym)[self.sym]['lastPrice']
        if self.out_the_money ^ (not IS_CALL):
            high_price=int(last_price*3)
            low_price = int(last_price) # adjust if bullish
        else:
            high_price=int(last_price+1)
            low_price = int(last_price)/2
            
        if IS_CALL:
            movement_price = last_price*1.15 # 1 + X*100 percent higher price used for influencing gain reward factor (using scholes).
        else:
            movement_price = last_price*.85
            
        print('movement_price', movement_price, last_price)
        
        def opt_price(row):
            return OptionPrice(float(movement_price), float(row['strikePrice']), int(row['daysToExpiration']/2) or 1, # days remaining
                                float(row['volatility']), RFR)
        
        scholes_gain = lambda row: (opt_price(row).call_price() if IS_CALL else opt_price(row).put_price())/row['last']
        
        # twenty percent gain implies this gain % on the option halfway to exp.
        try:
            df['volatility'] = df['volatility'].astype('float')
            if min(df['volatility']) < 0:
                df.drop(df.loc[df['volatility']==-999].index, inplace=True)
                df.loc[df['volatility'] < 0, 'volatility'] = df['volatility'].mean()
                print(df['volatility'].describe())
                print('Warning, replaced bad volatility value with mean')
            df['scholes'] = df.apply(scholes_gain, axis=1)
        except Exception as e:
            print('error getting scholes', e)
            print(df[['strikePrice', 'daysToExpiration', 'volatility', 'last']].describe())
            df['scholes'] = 0


        df['diff'] = abs(df['strikePrice'] - last_price)/last_price*100

        df = df[(df['strikePrice'] < high_price) & (df['strikePrice'] > low_price)]
        print(self.sym, low_price, high_price)
        # .sort_values(by='netCost', ascending=True)
        if self.buying:
            df = df[(df['ask'] < 10) & (df['ask'] > 1.25)]
        else:
            df = df[(df['ask'] > last_price*.02)] # want at least 2% of value. 
            
        df['leverage'] = abs((last_price*df['delta']) - df['mark']) / df['mark']
        
        try:
            next_earnings = datetime.fromtimestamp(yec.get_next_earnings_date(self.sym))
            print(self.sym, next_earnings)
            today = date.today()
            df['expire_date'] = pd.to_timedelta(df['daysToExpiration'], unit='d') + pd.to_datetime(today)
            # print(set(df['daysToExpiration']))
            df['covers_earnings'] = df['expire_date'] >= next_earnings
            print('next_earnings', next_earnings)
        except Exception as e:
            print('Unable to pull earnings', e)
            print('Invalid symbol', self.sym)
            df['covers_earnings'] = False

        # cost_per_dollar_day (lower is better, i.e. cost for an average % move per day)
        if self.out_the_money:
            df['cost_per_percent_day'] = df['last']/(df['daysToExpiration']/df['diff'])
        else:
            # Favor underpriced option cost (negative).
            time_cost = df['mark'] - abs(last_price - df['strikePrice'])
            df['cost_per_percent_day'] = (last_price + time_cost)/last_price

        df['score'] = df['cost_per_percent_day']/df['scholes']/(df['leverage']) # added leverage


        df = df[(df['last'] > df['bid']*.9) ^ (df['last'] > df['ask']*1) & (df['diff'] >= 1)]
     
        if self.buying:
            if IS_CALL:
                min_days = 210 if LONG_TERM else 20
                max_days = 900 if LONG_TERM else 210
            else:
                min_days = 0
                max_days = 30
        else:
            # desire last sale should be close to or lower than ask
            min_days = 0
            max_days = 120
            
        df = df[(df['last'] < 1.1*df['ask']) & (df['daysToExpiration'] <= max_days) & (df['daysToExpiration'] >= min_days)]
        print('days', min_days, max_days)
            
        self.df = df
#         return df

In [7]:
opts = []



In [8]:
# opt = Option(S, df)
# opt.info()


# SYM = 'TWLO

try:
    for s in SYMS:
        sym_df = c.optionsDF(s)
        opt = Option(s, sym_df, BUYING, OUT_THE_MONEY, OPTION_TYPE)
        opts.append(opt)
except Exception as e:
    traceback.print_exc()
    print(e)
    opts = []
    pass



name 'c' is not defined


Traceback (most recent call last):
  File "<ipython-input-8-7241bbfd3cfe>", line 9, in <module>
    sym_df = c.optionsDF(s)
NameError: name 'c' is not defined


In [9]:
if not opts:
# https://tdameritrad|e.readthedocs.io/en/latest/index.html#authentication
    res = td.auth.authentication(client_id='USS4AQEULHOTQVMHZPWG0JU6QHFPIUU7', redirect_uri='http://localhost:8080')
    init = True
    token = res['refresh_token'] # res['access_token']
    
    print('res', token, res)
# leave window open after authorizing

# df['expirationDate'].head()

KeyboardInterrupt: 

In [ ]:
# if chromedriver error:
# brew upgrade chromedriver # https://stackoverflow.com/questions/62072592/how-to-update-chromedriver-to-the-latest-version-on-macos-catalina-using-homebre


In [57]:
for o in opts:
    o.process_data()
    # o.info()

d 2020-05-01 01:00:00
(720, 49)
movement_price 55.332249999999995 48.115


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:68: RuntimeWarning: divide by zero encountered in double_scalars


FSLY 48 144
FSLY 2021-11-03 16:00:00
next_earnings 2021-11-03 16:00:00
days 0 120


In [58]:
# filter out bad volume

# df.shape

In [59]:
# p = OptionPrice(125, 125, 178, 41.67, .0061)
# p.call_price()

In [60]:
# pd.Timestamp(np.datetime64('2020-03-13 04:00:00'))
# df.info()

# diff is percent
DISPLAY_COLS = ['putCall','description', 'bid', 'ask', 'last', 'daysToExpiration', 'diff', 'covers_earnings', 'volatility', 'spread_ratio','totalVolume','scholes', 'leverage', 'percentChange', 'break_even','cost_per_percent_day','score']

def print_reduced(df):
    return df[DISPLAY_COLS]

df = pd.concat(opt.df for opt in opts)
df.shape

(23, 59)

In [61]:
SORT_COL = 'score'# "cost_per_percent_day"
# higher score is worse for purchase, better for selling.

res = df.sort_values(by=SORT_COL, ascending=BUYING).head(50)
print_reduced(res)

,putCall,description,bid,ask,last,daysToExpiration,diff,covers_earnings,volatility,spread_ratio,totalVolume,scholes,leverage,percentChange,break_even,cost_per_percent_day,score
81,CALL,FSLY Nov 12 2021 55 Call (Weekly),1.71,2.12,2.10,24,14.309467,True,84.873,0.193396,55,1.690476,6.843747,180.79,57.12,1.252078,0.108225
104,CALL,FSLY Nov 19 2021 60 Call,1.30,1.36,1.33,31,24.701237,True,83.276,0.044118,1285,1.488722,6.958872,216.67,61.36,1.059763,0.102295
103,CALL,FSLY Nov 19 2021 55 Call,2.23,2.30,2.27,31,14.309467,True,81.936,0.030435,5021,1.682819,6.132091,152.22,57.30,1.047822,0.101541
166,CALL,FSLY Jan 21 2022 60 Call,2.23,2.48,2.29,94,24.701237,True,60.900,0.100806,280,1.318777,4.973600,72.83,62.48,0.601764,0.091745
167,CALL,FSLY Jan 21 2022 65 Call,1.48,1.60,1.54,94,35.093006,True,60.830,0.075000,15,1.136364,5.561136,90.12,66.60,0.574928,0.090977
168,CALL,FSLY Jan 21 2022 70 Call,1.01,1.09,1.06,94,45.484776,True,61.733,0.073394,591,0.962264,6.011043,89.29,71.09,0.512913,0.088675
136,CALL,FSLY Dec 17 2021 60 Call,1.63,1.74,1.70,59,24.701237,True,66.420,0.063218,226,1.388235,6.089133,104.82,61.74,0.711731,0.084197
165,CALL,FSLY Jan 21 2022 55 Call,3.40,3.55,3.47,94,14.309467,True,60.238,0.042254,102,1.417867,4.449336,65.63,58.55,0.528232,0.083733
135,CALL,FSLY Dec 17 2021 55 Call,2.65,2.78,2.70,59,14.309467,True,65.718,0.046763,237,1.570370,5.350472,82.43,57.78,0.654840,0.077937
125,CALL,FSLY Nov 26 2021 60 Call (Weekly),1.22,1.40,1.30,38,24.701237,True,75.060,0.128571,9,1.561538,6.970195,165.31,61.40,0.845042,0.077639


In [62]:
set(o.df['volatility'])
o.df[['strikePrice','daysToExpiration','last','volatility']].dtypes

strikePrice         float64
daysToExpiration      int64
last                float64
volatility          float64
dtype: object